### Concrete understanding for bidirectional RNN (or GRU, LSTM)

#### I'm using PyTorch. Look at the [torch.nn.GRU](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU) docs. 

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
torch.manual_seed(0)

# sequence, batch, feature
random_input = Variable(torch.FloatTensor(5, 3, 4).normal_(), requires_grad=False)
random_input

tensor([[[-1.1258, -1.1524, -0.2506, -0.4339],
         [ 0.8487,  0.6920, -0.3160, -2.1152],
         [ 0.3223, -1.2633,  0.3500,  0.3081]],

        [[ 0.1198,  1.2377,  1.1168, -0.2473],
         [-1.3527, -1.6959,  0.5667,  0.7935],
         [ 0.5988, -1.5551, -0.3414,  1.8530]],

        [[ 0.7502, -0.5855, -0.1734,  0.1835],
         [ 1.3894,  1.5863,  0.9463, -0.8437],
         [-0.6136,  0.0316, -0.4927,  0.2484]],

        [[ 0.4397,  0.1124,  0.6408,  0.4412],
         [-0.1023,  0.7924, -0.2897,  0.0525],
         [ 0.5943,  1.5419,  0.5073, -0.5910]],

        [[-0.5692,  0.9200,  1.1108,  1.2899],
         [-1.4959, -0.1938,  0.4455,  1.3253],
         [-1.6293, -0.5497, -0.4798, -0.4997]]])

The bidirectional GRU has a forward part and a reverse part. They have separate weights, and for the reverse part, it looks from the end of the input sequence to the start of the sequence.


If we set the weights for the reverse GRU to the reverse part for bidirectional GRU, we should be able to see the same result when we apply the reverse GRU on a reversed input.

In [3]:
bidirectional_gru = torch.nn.GRU(input_size=4, hidden_size=3, num_layers=1, bidirectional=True)
unidirectional_gru = torch.nn.GRU(input_size=4, hidden_size=3, num_layers=1, bidirectional=False)

# Set the weights for the reverse unidirecetional_gru to be equal to the reverse weights of bidirectional gru
unidirectional_gru.weight_hh_l0 = bidirectional_gru.weight_hh_l0_reverse
unidirectional_gru.weight_ih_l0 = bidirectional_gru.weight_ih_l0_reverse
unidirectional_gru.bias_hh_l0 = bidirectional_gru.bias_hh_l0_reverse
unidirectional_gru.bias_ih_l0 = bidirectional_gru.bias_ih_l0_reverse

In [4]:
bi_output, bi_hidden = bidirectional_gru(random_input)
print(bi_output.shape)  # Size is: sequence, batch, n_directions * hidden_size
print(bi_hidden.shape)  # Size is: n_layers * n_directions, batch, hidden_size

# We run the unidirectional_gru on a reversed input
uni_output, uni_hidden = unidirectional_gru(random_input[np.arange(4, -1, -1), :, :])
print(uni_output.shape)  # Size is: sequence, batch, n_directions * hidden_size
print(uni_hidden.shape)  # Size is: n_layers * n_directions, batch, hidden_size

torch.Size([5, 3, 6])
torch.Size([2, 3, 3])
torch.Size([5, 3, 3])
torch.Size([1, 3, 3])


Here, the output sequence is organized as the last element of the sequence is the first element (since we reversed it). Each row is the hidden layer output for one example in the sequence.

In [5]:
uni_output[:, 0, :]

tensor([[ 0.1090, -0.4437, -0.0749],
        [-0.0774, -0.3606, -0.1737],
        [-0.3720, -0.1221, -0.3588],
        [-0.2162, -0.1459,  0.0202],
        [-0.5861,  0.0024, -0.3287]], grad_fn=<SelectBackward>)

For bidirectional GRU, the reversed part is the 2nd half of the output volume. 

Notice that the last element of the sequence corresponds to the first element in the uni_direcetional reversed GRU. This makes sense, as the output for the first element in the sequence would be the last element in the reversed GRU.

In [6]:
bi_output[:, 0, :] 

tensor([[-0.3761,  0.4892,  0.0113, -0.5861,  0.0024, -0.3287],
        [-0.0239,  0.0114, -0.0602, -0.2162, -0.1459,  0.0202],
        [ 0.0343, -0.0858,  0.0663, -0.3720, -0.1221, -0.3588],
        [-0.0130, -0.3720,  0.1178, -0.0774, -0.3606, -0.1737],
        [-0.2365, -0.6123,  0.3143,  0.1090, -0.4437, -0.0749]],
       grad_fn=<SelectBackward>)